In [3]:
# /Users/arshiabansal/Desktop/product-1211-sales-order-data.csv

In [4]:
import numpy as np
import pandas as pd

def tsl_monte_carlo(
    sales_order_file_path,
    initial_inventory=1000,
    holding_cost=1,
    ordering_cost=50,
    backorder_cost=5,
    lost_sale_cost=10,
    desired_service_level=0.95,
    MOQ=1000,
    today_date='2021-04-01',
    lead_time=4,
    total_days=100,
    review_period=7,
    num_simulations=1000
):
    # Read the sales order data
    demand_data = pd.read_csv(sales_order_file_path, index_col=0, parse_dates=True)

    # Rename the column if necessary
    if 'total qty' in demand_data.columns:
        demand_data.rename(columns={'total qty': 'total_qty'}, inplace=True)

    # Convert 'total_qty' to numeric and fill missing values
    demand_data['total_qty'] = pd.to_numeric(demand_data['total_qty'], errors='coerce').fillna(0)

    # Convert today's date to datetime
    today_date = pd.to_datetime(today_date)

    # Function to calculate safety stock
    def safety_stock_calculation(demand_std_dev, lead_time, Z):
        Z_score = abs(np.percentile(np.random.normal(size=1000000), 100 * Z))
        return Z_score * (lead_time ** 0.5) * demand_std_dev

    demand_std_dev = demand_data['total_qty'].std()
    safety_stock = safety_stock_calculation(demand_std_dev, lead_time, desired_service_level)

    # Calculate average forecasted demand
    avg_forecasted_demand = demand_data['total_qty'].mean()

    # Define range of max stock levels
    lower_bound = safety_stock
    upper_bound = safety_stock + 3 * avg_forecasted_demand  # Upper bound
    increment_step = avg_forecasted_demand / 2  # Example increment step

    max_stock_levels = np.arange(lower_bound, upper_bound + increment_step, increment_step).astype(int)

    # Function to calculate total cost
    def calculate_total_cost(df, holding_cost, ordering_cost, backorder_cost, lost_sale_cost):
        average_inventory = np.mean(df['stock_start'])
        total_orders = np.sum(df['order_placed'] > 0)
        total_holding_cost = average_inventory * holding_cost
        total_ordering_cost = total_orders * ordering_cost
        total_backorder_cost = np.sum(df['backorders']) * backorder_cost
        total_lost_sale_cost = np.sum(df['lost_sales']) * lost_sale_cost # to account for unfilled orders when determining optimal value of max IP
        return total_holding_cost + total_ordering_cost + total_backorder_cost + total_lost_sale_cost

    # Monte Carlo simulation function
    def monte_carlo_simulation(max_stock_level, num_simulations=1000):
        total_costs = []
        
        todays_index = demand_data.index.get_loc(today_date)
        end_index = todays_index + total_days
        
        for _ in range(num_simulations):
            # Initialize the DataFrame for the simulation
            data = {
                'day_tracker': pd.date_range(start=today_date, periods=total_days),
                'stock_start': np.zeros(total_days),
                'forecasted_demand': demand_data.iloc[todays_index:end_index]['total_qty'].values,
                'stock_end': np.zeros(total_days),
                'inventory_check': [False] * total_days,
                'order_placed': [False] * total_days,
                'orders_in_transit': [0] * total_days,
                'order_size': [0] * total_days,
                'backorders': np.zeros(total_days),
                'lost_sales': np.zeros(total_days)
            }
            
            df = pd.DataFrame(data)
            df.at[0, 'stock_start'] = initial_inventory
            
            # Simulation
            for i in range(total_days):
                if i > 0:
                    incoming_orders = df.at[i - lead_time, 'orders_in_transit'] if i - lead_time >= 0 else 0
                    df.at[i, 'stock_start'] = df.at[i - 1, 'stock_end'] + incoming_orders
                
                df.at[i, 'stock_end'] = df.at[i, 'stock_start'] - df.at[i, 'forecasted_demand']
                
                if df.at[i, 'stock_end'] < 0:
                    df.at[i, 'backorders'] = abs(df.at[i, 'stock_end'])
                    df.at[i, 'stock_end'] = 0
                
                if (i + 1) % review_period == 0:
                    df.at[i, 'inventory_check'] = True
                    
                    if df.at[i, 'stock_end'] < max_stock_level:
                        df.at[i, 'order_placed'] = True
                        order_size = max_stock_level - df.at[i, 'stock_end']
                        df.at[i, 'orders_in_transit'] = order_size
                        df.at[i, 'order_size'] = order_size
                
                # Check for lost sales
                if df.at[i, 'forecasted_demand'] > df.at[i, 'stock_start']:
                    df.at[i, 'lost_sales'] = df.at[i, 'forecasted_demand'] - df.at[i, 'stock_start']
            
            # Calculate total cost for the current simulation
            total_cost = calculate_total_cost(df, holding_cost, ordering_cost, backorder_cost, lost_sale_cost)
            total_costs.append(total_cost)
        
        # Return the average total cost for the given max_stock_level
        return np.mean(total_costs)

    # Run the Monte Carlo simulation for a range of max stock levels
    results = []
    for max_stock_level in max_stock_levels:
        avg_cost = monte_carlo_simulation(max_stock_level, num_simulations)
        results.append((max_stock_level, avg_cost))
        print(f"Max Stock Level: {max_stock_level}, Average Total Cost: {avg_cost}")

    # Convert results to a DataFrame for easy analysis
    results_df = pd.DataFrame(results, columns=['Max Stock Level', 'Average Total Cost'])

    # Find the optimal max stock level (the one with the minimum average total cost)
    optimal_max_stock_level = results_df.loc[results_df['Average Total Cost'].idxmin()]

    # Display the results
    print("\nOptimal Max Stock Level:")
    print(optimal_max_stock_level)
    print("\nAll Results:")
    print(results_df)

    return results_df, optimal_max_stock_level

# Example usage:
if __name__ == "__main__":
    sales_order_file_path = input("Please enter the path to the sales order data file: ")
    results_df, optimal_max_stock_level = tsl_monte_carlo(sales_order_file_path)
    print("\nOptimal Max Stock Level:")
    print(optimal_max_stock_level)
    print("\nAll Results:")
    print(results_df)


/var/folders/ns/1yxzfx892f36qy2_byrlq2kw0000gn/T/ipykernel_71097/3762062512.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  demand_data = pd.read_csv(sales_order_file_path, index_col=0, parse_dates=True)


Max Stock Level: 115, Average Total Cost: 43632.68
Max Stock Level: 143, Average Total Cost: 39036.60999999999
Max Stock Level: 172, Average Total Cost: 34920.75
Max Stock Level: 200, Average Total Cost: 31719.10999999999
Max Stock Level: 228, Average Total Cost: 28477.799999999992
Max Stock Level: 257, Average Total Cost: 25187.71
Max Stock Level: 285, Average Total Cost: 22257.23

Optimal Max Stock Level:
Max Stock Level         285.00
Average Total Cost    22257.23
Name: 6, dtype: float64

All Results:
   Max Stock Level  Average Total Cost
0              115            43632.68
1              143            39036.61
2              172            34920.75
3              200            31719.11
4              228            28477.80
5              257            25187.71
6              285            22257.23

Optimal Max Stock Level:
Max Stock Level         285.00
Average Total Cost    22257.23
Name: 6, dtype: float64

All Results:
   Max Stock Level  Average Total Cost
0           